In [1]:
import pandas as pd
import numpy as np

pd.set_option('max.columns', 131)

In [2]:
df = pd.read_csv('full_labels.csv').dropna(subset=['y'])

In [3]:
df.duplicated().mean()

0.0

In [4]:
df.shape

(297, 4)

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [6]:
df['upload_date'] = pd.to_datetime(df['upload_date'], format='%Y-%m-%d')
df['view_count'] = df['view_count'].astype(int)
df = df.sort_values('upload_date')
df_limpo = df.copy()
df_limpo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 297 entries, 151 to 34
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   title        297 non-null    object        
 1   upload_date  297 non-null    datetime64[ns]
 2   view_count   297 non-null    int64         
 3   y            297 non-null    int64         
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 11.6+ KB


In [7]:
features = pd.DataFrame(index=df_limpo.index)
y = df['y'].copy()

In [8]:
features['tempo_desde_pub'] = (pd.to_datetime('2021-01-31') - df_limpo['upload_date']) / np.timedelta64(1, 'D')
features['views'] = df_limpo['view_count']
features['views_por_dia'] = features['views'] / features['tempo_desde_pub']
features = features.drop(['tempo_desde_pub'], axis=1)

In [9]:
Xtrain, Xval = features.iloc[:140], features.iloc[140:]
ytrain, yval = y.iloc[:140], y.iloc[140:]
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((140, 2), (157, 2), (140,), (157,))

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

title_train = df_limpo.iloc[:140]['title']
title_val = df_limpo.iloc[140:]['title']

title_vec = TfidfVectorizer(min_df=1, ngram_range=(1, 3))
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)

In [11]:
title_bow_train.shape

(140, 2183)

In [12]:
from scipy.sparse import hstack

Xtrain_wtitle = hstack([Xtrain, title_bow_train])
Xval_wtitle = hstack([Xval, title_bow_val])

In [13]:
Xtrain_wtitle.shape, Xval_wtitle.shape

((140, 2185), (157, 2185))

# Random Forest

In [14]:
mdl = RandomForestClassifier(n_estimators=1000, random_state=0, min_samples_leaf=1, class_weight='balanced', n_jobs=3)
mdl.fit(Xtrain_wtitle, ytrain)

RandomForestClassifier(class_weight='balanced', n_estimators=1000, n_jobs=3,
                       random_state=0)

In [15]:
p = mdl.predict_proba(Xval_wtitle)[:, 1]

In [16]:
from sklearn.metrics import roc_auc_score, average_precision_score

In [17]:
print(f'avg={average_precision_score(yval, p)}, auc={roc_auc_score(yval, p)}')

avg=0.4452215654214563, auc=0.5782456140350878


# LightGBM

In [18]:
from lightgbm import LGBMClassifier

In [19]:
mdl = LGBMClassifier(random_state=0, class_weight='balanced', n_jobs=3)
mdl.fit(Xtrain_wtitle, ytrain)

LGBMClassifier(class_weight='balanced', n_jobs=3, random_state=0)

In [20]:
p = mdl.predict_proba(Xval_wtitle)[:, 1]

/home/gabriel/.anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:597: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


In [21]:
print(f'avg={average_precision_score(yval, p)}, auc={roc_auc_score(yval, p)}')

avg=0.40335269182299466, auc=0.5562280701754385


# Bayesian Optimization

In [22]:
from skopt import forest_minimize

In [23]:
def tune_lgbm(params):
    print(params)
    lr = params[0]
    max_depth = params[1]
    min_child_samples = params[2]
    subsample = params[3]
    colsample_bytree = params[4]
    n_estimators = params[5]
    min_df = params[6]
    ngram_range = (1, params[7])
    
    title_vec = TfidfVectorizer(min_df=min_df, ngram_range=ngram_range)
    title_bow_train = title_vec.fit_transform(title_train)
    title_bow_val = title_vec.transform(title_val)
    
    Xtrain_wtitle = hstack([Xtrain, title_bow_train])
    Xval_wtitle = hstack([Xval, title_bow_val])
    
    mdl = LGBMClassifier(
        learning_rate=lr,
        num_leaves=2 ** max_depth,
        max_depth=max_depth,
        min_child_samples=min_child_samples,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        bagging_freq=1,
        n_estimators=n_estimators,
        random_state=0,
        class_weight='balanced',
        n_jobs=3
    )
    
    mdl.fit(Xtrain_wtitle, ytrain)
    
    p = mdl.predict_proba(Xval_wtitle)[:, 1]
    
    print(roc_auc_score(yval, p))
    
    return -average_precision_score(yval, p)

space = [
    (1e-3, 1e-1, 'log-uniform'), # lr
    (1, 15), # max_depth
    (1, 20), # min_child_samples
    (0.05, 1.), # subsample
    (0.05, 1.), # colsample_bytree
    (100, 1000), # n_estimators
    (1, 5), # min_df
    (1, 5) # ngram_range
]

res = forest_minimize(tune_lgbm, space, random_state=160745, n_random_starts=20, n_calls=50, verbose=1)

Iteration No: 1 started. Evaluating function at random point.
[0.009944912110647982, 5, 1, 0.4677107511929402, 0.49263223036174764, 272, 3, 1]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


/home/gabriel/.anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:597: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


0.5378070175438597
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.5296
Function value obtained: -0.4142
Current minimum: -0.4142
Iteration No: 2 started. Evaluating function at random point.
[0.053887464791860025, 1, 15, 0.7437489153990157, 0.8675167974293533, 549, 3, 4]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


/home/gabriel/.anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:597: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')
/home/gabriel/.anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:597: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


0.5201754385964913
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.4221
Function value obtained: -0.3959
Current minimum: -0.4142
Iteration No: 3 started. Evaluating function at random point.
[0.004151454520895999, 6, 20, 0.8682075103820793, 0.9491436163200662, 411, 4, 3]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
0.5514035087719299
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.1958
Function value obtained: -0.3934
Current minimum: -0.4142
Iteration No: 4 started. Evaluating function at random point.
[0.0014099928811969545, 9, 9, 0.6502182010234373, 0.6866210554187129, 828, 5, 2]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


...

0.5320175438596491
Iteration No: 48 ended. Search finished for the next optimal point.
Time taken: 4.6141
Function value obtained: -0.3883
Current minimum: -0.4591
Iteration No: 49 started. Searching for the next optimal point.
[0.010942300203808828, 9, 14, 0.5414389443862576, 0.7555998291102568, 420, 5, 4]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
0.5570175438596491


/home/gabriel/.anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:597: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


Iteration No: 49 ended. Search finished for the next optimal point.
Time taken: 0.6909
Function value obtained: -0.4103
Current minimum: -0.4591
Iteration No: 50 started. Searching for the next optimal point.
[0.003429106657950565, 10, 20, 0.36099260125130456, 0.20334746014685123, 634, 4, 4]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
0.5353508771929825


/home/gabriel/.anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:597: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


Iteration No: 50 ended. Search finished for the next optimal point.
Time taken: 0.6634
Function value obtained: -0.4001
Current minimum: -0.4591
